In [1]:
#salut
import csv
data_encoding = 'utf-8'
data = []
with open('data/train.csv', 'r', encoding=data_encoding) as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

In [2]:
count = 0
for row in data:
    count += 1
    print(row[1])
    if count == 10:
        break

titles
L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.
Le maire de Bordeaux ne fait plus partie des Républicains et il tient à le montrer. Lors de ses voeux à la presse, l'édile a pris ses distances avec sa famille politique historique, qu'il trouve trop proche de Marine Le Pen.
En 2020, les tribunaux d'instance fusionnent avec ceux de grande instance pour former un unique "tribunal judiciaire". C'est la principale mesure de la réforme de la Justice, portée par la garde des sceaux Nicole Belloubet.
Les médecins jugés "gros prescripteurs d'arrêts maladie", sont notamment pointés du doigt par le patron du Medef Geoffroy Roux de Bézieux dans une interview au Figaro.
Il aura fallu mobiliser 90 gendarmes pour cette opération d'envergure menée en Gironde, dans le Var et dans les Pyrénées-Orientales mardi. Dix personnes ont été arrêtées, soupçonnées d'avoir participé à une escroquerie "d'ampleur" sur des voitures de luxe.
Les échanges de tirs ont

In [3]:
from operator import itemgetter
import pandas as pd
from rouge_score import rouge_scorer

# Read the validation data
validation_df = pd.read_csv('data/validation.csv')

rouge_scores = []
# Initialize Rouge Scorer
scorer = rouge_scorer.RougeScorer(['rougeL'])

# Function that generates summaries using LEAD-N
def lead_summary(text: pd.core.series.Series, titles: pd.core.series.Series, scorer: rouge_scorer.RougeScorer):
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        summaries.append([idx, sentences[0] + "."])
    return summaries

# Function that generates summaries using EXT-ORACLE
def ext_oracle_summary(text: pd.core.series.Series, titles: pd.core.series.Series, scorer: rouge_scorer.RougeScorer):
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        reference = titles.iloc[idx]
        rs = [scorer.score(sentence, reference)['rougeL'][2] for sentence in sentences]
        index, element = max(enumerate(rs), key=itemgetter(1))
        summaries.append([idx, sentences[index]])  
    return summaries
    
lead_summaries = lead_summary(validation_df['text'], validation_df['titles'], scorer)
ext_oracle_summaries = ext_oracle_summary(validation_df['text'], validation_df['titles'], scorer)

lead_rouge = []
ext_oracle_rouge = []
# Calculate the rouge-l score for each of the generated summaries compared to the original titles
for idx, title in validation_df['titles'].items():
    lead_rouge.append(scorer.score(lead_summaries[idx][1], title)['rougeL'][2])
    ext_oracle_rouge.append(scorer.score(ext_oracle_summaries[idx][1], title)['rougeL'][2])

avg_rouge_score_lead = sum(lead_rouge) / len(lead_rouge)
avg_rouge_score_ext_oracle = sum(ext_oracle_rouge) / len(ext_oracle_rouge)

print("Average Rouge-L F-Score with LEAD-1: ", avg_rouge_score_lead)
print("Average Rouge-L F-Score with EXT-ORACLE:", avg_rouge_score_ext_oracle)

# Store the generated summaries in the Kaggle-accepted format
lead_submission_df = pd.DataFrame(lead_summaries, columns=['ID', 'titles'])
ext_oracle_submission_df = pd.DataFrame(ext_oracle_summaries, columns=['ID', 'titles'])
lead_submission_df.to_csv('result/lead_submission.csv', index=False)
ext_oracle_submission_df.to_csv('result/ext_oracle_submission.csv', index=False)

Average Rouge-L F-Score with LEAD-1:  0.1535873817959459
Average Rouge-L F-Score with EXT-ORACLE: 0.31354067145919595


In [4]:
validation_df = pd.read_csv('data/test_text.csv')
def lead_summary(text: pd.core.series.Series):
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        summaries.append([idx, sentences[0] + "."])
    return summaries

lead_summaries = lead_summary(validation_df['text'])
lead_submission_df = pd.DataFrame(lead_summaries, columns=['ID', 'titles'])
lead_submission_df.to_csv('result/lead_submission_test.csv', index=False)

In [5]:
from sentence_transformers import SentenceTransformer

bert = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
import numpy as np

def compare_embeddings(text: str, sentences: list, model: SentenceTransformer):
    text_embedding = model.encode([text])
    sentence_embeddings = model.encode(sentences)
    similarities = np.dot(text_embedding[0], sentence_embeddings.T) / (np.linalg.norm(text_embedding) * np.linalg.norm(sentence_embeddings, axis=1))
    return similarities

def ext_abs_summary(text: pd.core.series.Series, model: SentenceTransformer, scorer: rouge_scorer.RougeScorer):
    summaries = []
    for idx, row in text.items():
        sentences = row.split(".")
        similarities = compare_embeddings(row, sentences, model)
        index, element = max(enumerate(similarities), key=itemgetter(1))
        summaries.append([idx, sentences[index]])
    return summaries


In [15]:
validation_df = pd.read_csv('data/validation.csv')
scorer = rouge_scorer.RougeScorer(['rougeL'])
ext_abs_summaries = ext_abs_summary(validation_df['text'], bert, scorer)
ext_abs_rouge = []
for idx, title in validation_df['titles'].items():
    ext_abs_rouge.append(scorer.score(ext_abs_summaries[idx][1], title)['rougeL'][2])

avg_rouge_score_ext_abs = sum(ext_abs_rouge) / len(ext_abs_rouge)
print("Average Rouge-L F-Score with EXT-ABS:", avg_rouge_score_ext_abs)

Average Rouge-L F-Score with EXT-ABS: 0.17171696791807112
